In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import json


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Froms
from src.gcpUtils.auth import getCredentials
from src.gcpUtils.bigQuery import pandasToBq, tableToPandas
from src.gcpUtils.google_storage_manager import *

cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")
# --- 1. Constantes de Negócio ---

# Caminho para seu arquivo de distâncias
ARQUIVO_DISTANCIAS = '../dados/distancias_todas_combinacoes.csv'

# Constantes para a fórmula de Leadtime Teórico
K = 5
p = 6
q = 4
C = 3
alpha = 1

ARQUIVO_JSON_FILIAIS = '../dados/filiais_inferior_30.json'
DATA_ANALISE = '2025-01-16'  # Formato AAAA-MM-DD
TABELA_BIGQUERY = 'planejamento-animale-292719.checklists_rollout.ANIMALE_checklist'
PATH_CREDENCAIS = '../bd/planejamento-animale-292719-296d49ccdea6.json'


In [ ]:

try:
    print(f"Carregando filiais do arquivo: {ARQUIVO_JSON_FILIAIS}...")
    
    with open(ARQUIVO_JSON_FILIAIS, 'r', encoding='utf-8') as f:
        dados_filiais = json.load(f)
        
    df_filiais = pd.DataFrame(dados_filiais)
    
    lista_nomes_filiais = df_filiais['FILIAL'].unique().tolist()
    
    if not lista_nomes_filiais:
        print("Atenção: Nenhuma filial foi encontrada no arquivo JSON.")
    else:
        print(f"Sucesso. {len(lista_nomes_filiais)} filiais únicas encontradas.")

except FileNotFoundError:
    print(f"Erro: Arquivo JSON não encontrado em '{ARQUIVO_JSON_FILIAIS}'")
    lista_nomes_filiais = []
except Exception as e:
    print(f"Erro ao ler o arquivo JSON: {e}")
    lista_nomes_filiais = []


# --- Montar e Executar a Query ---

if lista_nomes_filiais:
    
    # Ex: Transforma ['Filial A', 'Filial B'] em "('Filial A', 'Filial B')"
    filiais_para_query = "','".join(lista_nomes_filiais)
    filiais_para_query = f"('{filiais_para_query}')"

    query = f"""
        SELECT SKU, FILIAL, VELOCIDADE_VENDA, ALVO, PRESENTE, TRANSITO, 
        EST_TOTAL, CONT_RUPTURA, CONT_FALTA, CONT_EXCESSO, VOLUME_EXCESSO, REGULADOR
        FROM {TABELA_BIGQUERY}
        WHERE FILIAL IN {filiais_para_query}
        AND DATA = '{DATA_ANALISE}'
    """
    try:
        # --- Buscar os dados ---
        
        print("Executando consulta no BigQuery...")
        df_resultado = tableToPandas(query, 'planejamento-animale-292719', cred)
        
        print("\n--- Resultado da Consulta ---")
        if df_resultado.empty:
            print("A consulta não retornou dados.")
        else:
            print(df_resultado)

    except Exception as e:
        print(f"\nErro ao executar a consulta no BigQuery: {e}")

else:
    print("\nAnálise não executada pois nenhuma filial foi carregada.")

In [ ]:
def carregar_matriz_distancia(caminho_csv):
    """Carrega o CSV de distâncias."""
    print(f"Carregando matriz de distâncias de: {caminho_csv}...")
    try:
        df_dist = pd.read_csv(
            caminho_csv,
            usecols=['Filial_A', 'Filial_B', 'Distancia_km'] 
        )
        

        print(f"Matriz de distâncias carregada com {len(df_dist)} combinações.")
        return df_dist

    except FileNotFoundError:
        print(f"Erro: Arquivo CSV não encontrado em '{caminho_csv}'")
        return pd.DataFrame(columns=['Filial_A', 'Filial_B', 'Distancia_km'])
    except Exception as e:
        print(f"Erro ao carregar CSV: {e}")
        return pd.DataFrame(columns=['Filial_A', 'Filial_B', 'Distancia_km'])


# --- Lógica Principal de Ranking ---

def calcular_ranking_transferencia(df_estoque, df_distancias, filial_fonte_escolhida):
    """
    Calcula o ranking de transferência a partir de UMA ÚNICA filial fonte
    para todas as outras lojas com demanda, usando a nova fórmula.
    """
    filial_fonte_escolhida = filial_fonte_escolhida.strip()

    fator_preferencia = alpha if filial_fonte_escolhida == "CENTRO DE DISTRIBUICAO" else 1
    
    if df_estoque.empty or df_distancias.empty:
        print("Erro: Dados de estoque ou distâncias estão vazios. Encerrando.")
        return pd.DataFrame()

    # print(f"\nIniciando cálculo do ranking a partir da FONTE: {filial_fonte_escolhida}...")

    # Renomeia 'PRESENTE' para 'EST_DISP' (Estoque Disponível)
    if 'PRESENTE' in df_estoque.columns:
        df_estoque = df_estoque.rename(columns={'PRESENTE': 'EST_DISP'})
    
    # --- Cálculo da Demanda Líquida ---
    df_estoque['DEMANDA_LIQUIDA'] = np.maximum(0, df_estoque['ALVO'].fillna(0) - df_estoque['EST_TOTAL'].fillna(0))
    
    # --- Identificando Fonte e Destinos ---
    df_fonte_data = df_estoque[
        df_estoque['FILIAL'] == filial_fonte_escolhida
    ][['SKU', 'VOLUME_EXCESSO']]
    
    if df_fonte_data.empty:
        print(f"Atenção: A filial fonte '{filial_fonte_escolhida}' não foi encontrada nos dados de estoque.")
    
    df_destinos = df_estoque[
        ( (df_estoque['DEMANDA_LIQUIDA'] > 0) | (df_estoque['CONT_RUPTURA'] == 1) ) &
        ( df_estoque['FILIAL'] != filial_fonte_escolhida )
    ][[
        'SKU', 'FILIAL', 'DEMANDA_LIQUIDA', 
        'CONT_RUPTURA', 'CONT_FALTA',
        'VELOCIDADE_VENDA'
    ]].rename(columns={'FILIAL': 'FILIAL_DESTINO'})
    
    if df_destinos.empty:
        print("Nenhuma loja de DESTINO (com demanda) encontrada.")
        return pd.DataFrame()

    # --- Pares de Transferência ---
    # print("  Combinando pares de transferência (Fonte -> Destino)...")
    
    df_pares = pd.merge(
        df_destinos,
        df_fonte_data,
        on='SKU',
        how='left' 
    )
    
    df_pares['FILIAL_FONTE'] = filial_fonte_escolhida
    
    # Preenche 'VOLUME_EXCESSO' com 0 se a fonte não tiver o SKU
    df_pares['VOLUME_EXCESSO'] = df_pares['VOLUME_EXCESSO'].fillna(0)

    # --- Juntar Distâncias ---

    df_distancias_fonte = df_distancias[
        df_distancias['Filial_A'] == filial_fonte_escolhida
    ].copy()
    
    # Junta os pares com as distâncias filtradas
    df_ranking = pd.merge(
        df_pares, 
        df_distancias_fonte,
        left_on=['FILIAL_FONTE', 'FILIAL_DESTINO'], 
        right_on=['Filial_A', 'Filial_B']
    )
    
    if df_ranking.empty:
        print(f"Erro: Não foram encontradas distâncias de '{filial_fonte_escolhida}' para os destinos.")
        print("Verifique seu CSV de distâncias.")
        return pd.DataFrame()
    
    # --- Cálculo dos Fatores da Fórmula ---
    
    df_ranking['LEADTIME'] = C * df_ranking['Distancia_km']
    df_ranking['CUSTO_FRETE'] = K * df_ranking['Distancia_km']
    
    fator_logistica = (1 / (df_ranking['LEADTIME'] + 1)) + (1 / (df_ranking['CUSTO_FRETE'] + 1))

    cont_ruptura = df_ranking['CONT_RUPTURA'].fillna(0)
    cont_falta = df_ranking['CONT_FALTA'].fillna(0)
    
    fator_demanda = (cont_ruptura * p) + (cont_falta * q)

    velocidade_venda = df_ranking['VELOCIDADE_VENDA'].fillna(0)
    
    # --- Ranking Bruto ---
    df_ranking['RANKING_TRANSFERENCIA_BRUTO'] = velocidade_venda * fator_logistica * fator_demanda * fator_preferencia
    
    df_ranking['QTD_A_TRANSFERIR'] = np.minimum(
        df_ranking['VOLUME_EXCESSO'], df_ranking['DEMANDA_LIQUIDA']
    )
    
    df_ranking['RANKING_TRANSFERENCIA'] = df_ranking['RANKING_TRANSFERENCIA_BRUTO']
    df_ranking.loc[df_ranking['QTD_A_TRANSFERIR'] == 0, 'RANKING_TRANSFERENCIA'] = 0

    # --- Limpar e Ordenar ---
    df_resultado = df_ranking.sort_values(by='RANKING_TRANSFERENCIA', ascending=False)
    
    # Colunas de interesse para o relatório final (ATUALIZADAS)
    colunas_finais = [
        'SKU', 
        'FILIAL_FONTE', 
        'FILIAL_DESTINO', 
        'RANKING_TRANSFERENCIA',
        'QTD_A_TRANSFERIR',
        'VOLUME_EXCESSO',
        'DEMANDA_LIQUIDA',
        'Distancia_km',
        'LEADTIME',         
        'CUSTO_FRETE',      
        'CONT_RUPTURA',     
        'CONT_FALTA',       
    ]
    
    # Quantidade de SKUs sugeridos para transferir
    # print(f"  Total de SKUs sugeridos para transferência: {df_resultado[df_resultado['RANKING_TRANSFERENCIA'] > 0].shape[0]}")
    
    return df_resultado[colunas_finais].reset_index(drop=True)

def calcular_ranking_loop_nxn(df_estoque_completo, df_distancias_total, alpha, C, K, p, q):
    
    filiais_fonte_unicas = df_estoque_completo[
        df_estoque_completo['VOLUME_EXCESSO'].fillna(0) > 0
    ]['FILIAL'].unique()
    
    lista_de_rankings = [] # Lista para guardar os DataFrames de cada filial
    
    # Loop principal: chama a função 1-para-N para cada filial
    for filial_fonte in filiais_fonte_unicas:
        
        # Chama a sua função já implementada
        df_ranking_da_filial = calcular_ranking_transferencia(
            df_estoque_completo,
            df_distancias_total,
            filial_fonte  # <- Passa a filial atual do loop
        )
        
        # Adiciona o resultado (mesmo que vazio) à lista
        if not df_ranking_da_filial.empty:
            lista_de_rankings.append(df_ranking_da_filial)

    # --- Consolidar e Retornar o Resultado ---
    if not lista_de_rankings:
        print("\nNenhuma sugestão de transferência foi gerada em toda a rede.")
        return pd.DataFrame()
    else:
        df_ranking_final_global = pd.concat(lista_de_rankings, ignore_index=True)
        
        # Ordena o ranking mestre pela maior prioridade
        df_ranking_final_global = df_ranking_final_global.sort_values(
            by='RANKING_TRANSFERENCIA', ascending=False
        )
        
        return df_ranking_final_global

# --- 4. Execução Principal ---
if __name__ == "__main__":
    
    FILIAL_FONTE_ESCOLHIDA = "CENTRO DE DISTRIBUICAO".strip()

    
    # 1. Carregar dados do banco (ou simulação)
    df_estoque_dia = df_resultado.copy()
    # 2. Carregar matriz de distâncias
    df_distancias_total = carregar_matriz_distancia(ARQUIVO_DISTANCIAS)

    # 1. Agrupa por SKU e pega o primeiro valor de REGULADOR (deve ser igual para todos)
    df_cd_estoque = df_estoque_dia.groupby('SKU', as_index=False)['REGULADOR'].first()
    
    # 2. Define a FILIAL
    df_cd_estoque['FILIAL'] = "CENTRO DE DISTRIBUICAO".strip()
    
    # 3. Renomeia REGULADOR para VOLUME_EXCESSO (para que a função de ranking entenda)
    df_cd_estoque = df_cd_estoque.rename(columns={'REGULADOR': 'VOLUME_EXCESSO'})
    
    # 4. Concatena o DataFrame original (lojas) com o novo (CD)
    df_estoque_completo = pd.concat([df_estoque_dia, df_cd_estoque], ignore_index=True)
    
    print(f"Estoque do CD criado para {len(df_cd_estoque)} SKUs.")
    print(f"DataFrame total agora tem {len(df_estoque_completo)} linhas.")

    # 3. Calcular o ranking
    df_ranking_final = calcular_ranking_transferencia(
        df_estoque_completo, 
        df_distancias_total,
        FILIAL_FONTE_ESCOLHIDA
    )
    
    # 4. Exibir o resultado
    # if not df_ranking_final.empty:
    #     print(f"\n--- RANKING DE TRANSFERÊNCIA (FONTE: {FILIAL_FONTE_ESCOLHIDA}) ---")
    #     print(df_ranking_final.to_markdown(index=False, floatfmt=".4f"))

    # else:
    #     print("\nNenhuma sugestão de transferência foi gerada.")

In [ ]:
df_ranking_final = calcular_ranking_loop_nxn(
        df_estoque_completo,
        df_distancias_total,
        alpha, C, K, p, q
    )

if not df_ranking_final.empty:
        print("\n--- RANKING DE TRANSFERÊNCIA GLOBAL (N x N) ---")
        print(df_ranking_final.to_markdown(index=False, floatfmt=".4f"))
        # resultados relativos ao SKU 25.34.1671-0005-TAM_2
        print("")
else:
        print("\nNenhuma sugestão de transferência foi gerada.")

In [ ]:
# --- Análise Específica do SKU ---

# 1. Defina o SKU que você quer analisar
sku_desejado = "25.34.1671-0005-TAM_2".strip()

# 2. Verifique se o DataFrame final existe e não está vazio
#    (Use 'df_ranking_final' ou 'df_ranking_final_global', 
#     dependendo de como você chamou a variável)
if 'df_ranking_final' in locals() and not df_ranking_final.empty:
    
    # 3. Filtre o DataFrame de ranking
    df_sku_filtrado = df_ranking_final[
        df_ranking_final['SKU'].astype(str).str.strip() == sku_desejado
    ]

    # 4. Imprima os resultados para esse SKU
    if df_sku_filtrado.empty:
        print(f"\nNenhum ranking de transferência encontrado para o SKU: {sku_desejado}")
    else:
        print(f"\n--- Ranking de Transferência para o SKU: {sku_desejado} ---")
        print(df_sku_filtrado.to_markdown(index=False, floatfmt=".4f"))
        
else:
    print("\nErro: O DataFrame 'df_ranking_final' não foi encontrado ou está vazio.")
    print("Por favor, execute a célula de cálculo do ranking primeiro.")

# Nome do arquivo de saída
nome_arquivo_excel = "ranking_global_transferencias.xlsx"

try:
    # (Use 'df_ranking_final' ou 'df_ranking_final_global',
    #  qualquer que seja o nome da sua variável final)
    
    print(f"Salvando o ranking completo em '{nome_arquivo_excel}'...")
    
    df_ranking_final.to_excel(
        nome_arquivo_excel, 
        sheet_name="Ranking_Completo",
        index=False  # Importante: para não salvar o índice 0, 1, 2... do pandas
    )
    
    print(f"Sucesso! Arquivo salvo.")
    
except NameError:
    print("Erro: O DataFrame 'df_ranking_final' não foi encontrado.")
except Exception as e:
    print(f"Erro ao salvar o arquivo: {e}")